In [1]:
# Import libraries
import numpy as np
import pandas as pd
import geopandas as gpd
import libpysal as ps
import matplotlib.pyplot as plt
import matplotlib as mpl

#For basemap:
import contextily as cx

#For regression:
import statsmodels.api as sm
import numpy as np
from patsy import dmatrices

import esda as esda
from esda.moran import Moran_Local
import splot as splot

#Plot spatial weights:
from libpysal.weights.contiguity import Queen
from splot.libpysal import plot_spatial_weights
from splot.esda import moran_scatterplot, plot_moran
from splot.esda import plot_local_autocorrelation

# For GWR
from mgwr.gwr import GWR, MGWR
from mgwr.sel_bw import Sel_BW
from mgwr.utils import compare_surfaces, truncate_colormap
import os

/Users/meera/opt/anaconda3/lib/python3.8/site-packages/geopandas/_compat.py:111: UserWarning: The Shapely GEOS version (3.10.2-CAPI-1.16.0) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(
/Users/meera/opt/anaconda3/lib/python3.8/site-packages/statsmodels/tsa/base/tsa_model.py:7: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import (to_datetime, Int64Index, DatetimeIndex, Period,
/Users/meera/opt/anaconda3/lib/python3.8/site-packages/statsmodels/tsa/base/tsa_model.py:7: FutureWarning: pandas.Float64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import (to_datetime, Int64Index, DatetimeIndex, Period,


In [15]:
#read in the the full data 
os.chdir('/Users/meera/Documents/drugs/Data')
od_geometry = pd.read_csv('od_final.csv')

In [16]:
od_geometry

,Year,FIPS,County,State,Region,Division,Overdose_Deaths,Overdose_Pop,Overdose_Rate_per_100k,Population,...,Opioid_Disp_Rate_per_100k,Pct_Unemployed,Pct_Poverty,Median_HHI,Jail_Pop,Urbanicity,Spatial_Max,Spatial_Mean,Prev_Spatial_Max,Prev_Spatial_Mean
0,2011,1001,"Autauga County, AL",Alabama,South,East South Central,NaN,NaN,NaN,50756.0,...,144100.0,8.3,14.9,48863.0,161.00,2.0,NaN,NaN,25.204500,16.526161
1,2011,1003,"Baldwin County, AL",Alabama,South,East South Central,24.0,186717.0,12.853677,179878.0,...,150000.0,9.0,13.4,50144.0,743.36,2.0,42.000263,21.667119,26.096714,16.138937
2,2011,1005,"Barbour County, AL",Alabama,South,East South Central,NaN,NaN,NaN,29737.0,...,99400.0,11.4,29.5,30117.0,105.00,1.0,NaN,NaN,NaN,NaN
3,2011,1007,"Bibb County, AL",Alabama,South,East South Central,NaN,NaN,NaN,21587.0,...,57700.0,10.4,22.2,37347.0,67.50,3.0,13.640773,12.565340,25.204500,15.378684
4,2011,1009,"Blount County, AL",Alabama,South,East South Central,NaN,NaN,NaN,58345.0,...,64800.0,8.6,14.9,41940.0,127.50,3.0,55.504718,23.305981,50.728854,22.318995
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31415,2020,56037,"Sweetwater County, WY",Wyoming,West,Mountain,15.0,42673.0,35.151032,43051.0,...,70000.0,7.4,7.6,70583.0,75.00,1.0,NaN,NaN,28.017626,28.017626
31416,2020,56039,"Teton County, WY",Wyoming,West,Mountain,NaN,NaN,NaN,23081.0,...,50600.0,6.0,5.2,92488.0,25.00,1.0,18.013002,18.013002,28.017626,20.953692
31417,2020,56041,"Uinta County, WY",Wyoming,West,Mountain,NaN,NaN,NaN,20299.0,...,60000.0,6.3,8.5,71246.0,55.00,1.0,35.151032,35.151032,NaN,NaN
31418,2020,56043,"Washakie County, WY",Wyoming,West,Mountain,NaN,NaN,NaN,7885.0,...,40200.0,5.3,9.7,58532.0,28.00,1.0,19.798305,19.798305,28.017626,22.148260


In [18]:
# read in the polygon data (found from census)
os.chdir('/Users/meera/Documents/drugs/Data/Geographical_Data/cb_2018_us_county_20m')
gdf = gpd.read_file('cb_2018_us_county_20m.shp')
gdf['FIPS'] = gdf['STATEFP'] + gdf['COUNTYFP']
gdf = gdf[['FIPS', 'geometry']]

In [19]:
gdf

,FIPS,geometry
0,37017,"POLYGON ((-78.90200 34.83527, -78.79960 34.850..."
1,37167,"POLYGON ((-80.49737 35.20210, -80.29542 35.502..."
2,39153,"POLYGON ((-81.68699 41.13596, -81.68495 41.277..."
3,42113,"POLYGON ((-76.81373 41.59003, -76.22014 41.541..."
4,48459,"POLYGON ((-95.15274 32.66095, -95.15211 32.902..."
...,...,...
3215,38005,"POLYGON ((-99.84661 48.37130, -99.49292 48.370..."
3216,72079,"POLYGON ((-67.10904 18.05608, -67.06791 18.066..."
3217,31159,"POLYGON ((-97.36812 41.04695, -96.91094 41.046..."
3218,37023,"POLYGON ((-81.90665 35.88338, -81.94319 35.960..."


In [20]:
# TURN FIPS COL TO STRING 
od_geometry['FIPS'] = od_geometry['FIPS'].astype(str).str.zfill(5)
od_geometry

,Year,FIPS,County,State,Region,Division,Overdose_Deaths,Overdose_Pop,Overdose_Rate_per_100k,Population,...,Opioid_Disp_Rate_per_100k,Pct_Unemployed,Pct_Poverty,Median_HHI,Jail_Pop,Urbanicity,Spatial_Max,Spatial_Mean,Prev_Spatial_Max,Prev_Spatial_Mean
0,2011,01001,"Autauga County, AL",Alabama,South,East South Central,NaN,NaN,NaN,50756.0,...,144100.0,8.3,14.9,48863.0,161.00,2.0,NaN,NaN,25.204500,16.526161
1,2011,01003,"Baldwin County, AL",Alabama,South,East South Central,24.0,186717.0,12.853677,179878.0,...,150000.0,9.0,13.4,50144.0,743.36,2.0,42.000263,21.667119,26.096714,16.138937
2,2011,01005,"Barbour County, AL",Alabama,South,East South Central,NaN,NaN,NaN,29737.0,...,99400.0,11.4,29.5,30117.0,105.00,1.0,NaN,NaN,NaN,NaN
3,2011,01007,"Bibb County, AL",Alabama,South,East South Central,NaN,NaN,NaN,21587.0,...,57700.0,10.4,22.2,37347.0,67.50,3.0,13.640773,12.565340,25.204500,15.378684
4,2011,01009,"Blount County, AL",Alabama,South,East South Central,NaN,NaN,NaN,58345.0,...,64800.0,8.6,14.9,41940.0,127.50,3.0,55.504718,23.305981,50.728854,22.318995
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31415,2020,56037,"Sweetwater County, WY",Wyoming,West,Mountain,15.0,42673.0,35.151032,43051.0,...,70000.0,7.4,7.6,70583.0,75.00,1.0,NaN,NaN,28.017626,28.017626
31416,2020,56039,"Teton County, WY",Wyoming,West,Mountain,NaN,NaN,NaN,23081.0,...,50600.0,6.0,5.2,92488.0,25.00,1.0,18.013002,18.013002,28.017626,20.953692
31417,2020,56041,"Uinta County, WY",Wyoming,West,Mountain,NaN,NaN,NaN,20299.0,...,60000.0,6.3,8.5,71246.0,55.00,1.0,35.151032,35.151032,NaN,NaN
31418,2020,56043,"Washakie County, WY",Wyoming,West,Mountain,NaN,NaN,NaN,7885.0,...,40200.0,5.3,9.7,58532.0,28.00,1.0,19.798305,19.798305,28.017626,22.148260


In [21]:
# MERGE 
od = od_geometry.merge(gdf, on = ['FIPS'], how = 'left')

In [22]:

od['log_Pct_Black'] = np.log(od['Pct_Black']+1)
od['log_PrimCarePhys_per_100k'] = np.log(od['PrimCarePhys_per_100k'])
od['log_Spatial_Mean'] = np.log(od['Spatial_Mean'])
od['log_Overdose_Rate_per_100k'] = np.log(od['Overdose_Rate_per_100k'])
od = od[['Year', 'State', 'log_Overdose_Rate_per_100k', 'Pct_Age_lt_18', 'log_Pct_Black', 'Potential_Years_Lost', 'Pct_Uninsured',
         'log_PrimCarePhys_per_100k', 'Pct_Child_in_1ParentHH', 'Pct_Poverty', 'log_Spatial_Mean', 'geometry']]
od = od.dropna()
od = od.reset_index(drop=True)



/Users/meera/opt/anaconda3/lib/python3.8/site-packages/pandas/core/arraylike.py:397: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [23]:
geometry = od['geometry']
geometry.dtypes

In [24]:
od = gpd.GeoDataFrame(od, geometry = geometry)
od.to_file('geo_od_nonull.geojson', driver='GeoJSON')

/Users/meera/opt/anaconda3/lib/python3.8/site-packages/geopandas/io/file.py:362: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  pd.Int64Index,


In [25]:
os.getcwd()

'/Users/meera/Documents/drugs/Data/Geographical_Data/cb_2018_us_county_20m'